In [20]:
import requests
import pandas as pd
import numpy as np
import math
import json
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from IPython.display import Markdown as md
import os

output_notebook()
url = "http://bing.com/covid/data"
r = requests.get(url)
j = r.json()
with open('state_populations.json') as data:
    state_pop = pd.read_json(data)
#j = r.json()
#print(j)
#df = pd.DataFrame(j['areas'][2])
df = pd.json_normalize(j['areas'][2], 'areas')
#df['totalConfirmed'].sum()
index = df['displayName'].to_list()
totalConfirmed = df['totalConfirmed'].to_list()
totalDeaths = df['totalDeaths'].to_list()
totalRecovered = df['totalRecovered'].to_list()
df2 = pd.DataFrame({'totalConfirmed': totalConfirmed,
                    'totalDeaths': totalDeaths,
                    'totalRecovered': totalRecovered}, index=index)
df2['totalRecovered'].fillna(0, inplace=True)
df2['deathRate'] = df2['totalDeaths'].div(df2['totalConfirmed'], fill_value=0)
df2['recoveryRate'] = df2['totalRecovered'].div(df2['totalConfirmed'], fill_value=0)
covid = pd.merge(left=df2, right=state_pop[['state_name','population']], how='left', left_index=True, right_on='state_name')
covid['infectionRate'] = covid['totalConfirmed'].div(covid['population'], fill_value=0)
covid = covid[['state_name', 'population', 'totalConfirmed', 'infectionRate', 'totalDeaths', 'deathRate', 'totalRecovered', 'recoveryRate']]
covid.set_index('state_name', inplace=True)

total_population = int(covid['population'].sum())
total_confirmed = int(covid['totalConfirmed'].sum())
total_deaths = int(covid['totalDeaths'].sum())
total_recoveries = int(covid['totalRecovered'].sum())
infection_rate = total_confirmed / total_population
death_rate = total_deaths / total_confirmed
recovery_rate = total_recoveries / total_confirmed

covid['expected'] = covid['population'].mul(infection_rate)
covid['delta'] = covid['totalConfirmed'].sub(covid['expected'])
covid['deltaRate'] = covid['totalConfirmed'].div(covid['delta'])

Loading BokehJS ...

In [23]:
var_list = [covid.loc['Tennessee', 'infectionRate']]           # TN Infection Rate
var_list.append(infection_rate)                                # US Infection Rate
var_list.append(covid.loc['Tennessee', 'deathRate'])           # TN Death Rate
var_list.append(death_rate)                                    # US Death Rate
var_list.append(int(covid.loc['Tennessee', 'delta']))          # TN Infection Delta
var_list.append(covid.loc['Tennessee', 'deltaRate'])           # TN Delta Rate
var_list.append(int(covid.loc['Tennessee', 'expected']))       # TN Expected Cases
var_list.append(int(total_confirmed))                          # US Expected Cases = Total Confirmed
var_list.append(int(covid.loc['Tennessee', 'totalConfirmed'])) # TN Actual Cases
var_list.append(int(total_confirmed))                          # US Actual Cases = Total Confirmed

table = """ | Metric | `Tennessee` | `United States` |
| --- | ---: | ---: |
| Infection Rate | `{:.4%}` | `{:.4%}` |
| Death Rate | `{:.4%}` | `{:.4%}` |
| Infection Delta | `{:,}` | `-----` |
| Delta % | `{:.4%}` | `-----` |
| Expected | `{:,}` | `{:,}` |
| Actual | `{:,}` | `{:,}` |
""".format(*var_list)

#tennessee = """**`Tennessee:`**  
#Infection rate is **`{:.4%}`**   
#Death rate is **`{:.4%}`**   
#Actual/expected delta is **`{:,}`** or **`{:.4%}`**  
#Expected Infections: **`{:,}`**  
#Actual Infections: **`{:,}`**
#""".format(*var_list)

#tennessee = "**`Tennessee`**:  "
#tennessee += "The current infection rate is **`{:.4%}`**.  ".format(covid.loc['Tennessee', 'infectionRate'])
#tennessee += "The current death rate is **`{:.4%}`**.  ".format(covid.loc['Tennessee', 'deathRate'])
#tennessee += "The current actual/expected delta is **`{:,}`** or **`{:.4%}`**.  ".format(int(covid.loc['Tennessee', 'delta']), covid.loc['Tennessee', 'deltaRate'])
md(table)

 | Metric | `Tennessee` | `United States` |
| --- | ---: | ---: |
| Infection Rate | `0.0114%` | `0.0195%` |
| Death Rate | `0.3827%` | `1.4117%` |
| Infection Delta | `-562` | `-----` |
| Delta % | `-139.3984%` | `-----` |
| Expected | `1,346` | `64,674` |
| Actual | `784` | `64,674` |


In [22]:
source = ColumnDataSource(data=covid)
width = 0.5

title_p = "COVID Confirmed: {:,} - Death Rate: {:.4%} - Recovery Rate: {:.4%}".format(total_confirmed, death_rate, recovery_rate)
title_q = "COVID Deaths: {:,} - Death Rate: {:.4%}".format(total_deaths, death_rate)
title_z = "COVID Recovered: {:,} - Recovery Rate: {:.4%}".format(total_recoveries, recovery_rate)
title_x = "COVID US National Infection Rate: {:.4%}".format(infection_rate)
title_y = "COVID US Confirmed / Expected Delta"

p = figure(x_range=index, plot_height=800, plot_width=1500, title=title_p)
p.vbar(x='state_name', top='totalConfirmed', width=width, source=source, line_color='white')
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.y_range.end = covid['totalConfirmed'].max() + 1000
p.xaxis.major_label_orientation = math.pi / 4

q = figure(x_range=index, plot_height=800, plot_width=1500, title=title_q)
q.vbar(x='state_name', top='totalDeaths', width=width, source=source, line_color='white')
q.xgrid.grid_line_color = None
q.y_range.start = 0
q.y_range.end = covid['totalDeaths'].max() + 100
q.xaxis.major_label_orientation = math.pi / 4

z = figure(x_range=index, plot_height=800, plot_width=1500, title=title_z)
z.vbar(x='state_name', top='totalRecovered', width=width, source=source, line_color='white')
z.xgrid.grid_line_color = None
z.y_range.start = 0
z.y_range.end = covid['totalRecovered'].max() + 50
z.xaxis.major_label_orientation = math.pi / 4

x = figure(x_range=index, plot_height=800, plot_width=1500, title=title_x)
x.vbar(x='state_name', top='infectionRate', width=width, source=source)
x.xgrid.grid_line_color = None
x.y_range.start = 0
x.y_range.end = covid['infectionRate'].max() + 0.0005
x.xaxis.major_label_orientation = math.pi / 4

y = figure(x_range=index, plot_height=800, plot_width=1500, title=title_y)
y.vbar(x='state_name', top='delta', width=width, source=source)
y.xgrid.grid_line_color = None
y.y_range.start = covid['delta'].min() - 500
y.y_range.end = covid['delta'].max() + 500
y.xaxis.major_label_orientation = math.pi / 4

show(x)
show(y)
show(p)
show(q)
show(z)